In [153]:
import dbstep.Dbstep as db
    
    #Create DBSTEP object
#mol = db.dbstep("xyz_files/C=C1CCCC1=O.xyz",atom1=1,atom2=2,commandline=True,verbose=True,sterimol=True,measure='classic')  
    
    #Grab Sterimol Parameters
L = mol.L
Bmin = mol.Bmin
Bmax = mol.Bmax

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import os 
input_file = 'sterimol/IDPiDataset_noI.xlsx'

df1 = pd.read_excel(input_file)

electrophileSMILES = df1['electrophile SMILES'].values
nucleophileSMILES = df1['nucleophile SMILES'].values
# Specify the column number (0-based index) containing strings
# Read the CSV file into a DataFrame
catSMILES = df1['3,3â€™ Catalyst Substituent '].values

def smiles_to_xyz(smiles, output_file):
    # Handle cases where there is a '/' in the SMILES string
    

    mol = Chem.MolFromSmiles(smiles)
    
    if mol is not None:
        # Add hydrogen atoms
        mol = Chem.AddHs(mol)
        
        # Generate 3D coordinates
        AllChem.EmbedMolecule(mol)
        
        # Write coordinates and SMILES to .xyz file
        with open(output_file, 'w') as xyz_file:
            xyz_file.write(f"{mol.GetNumAtoms()}\n")
            xyz_file.write(f"{smiles}\n")

            for atom in mol.GetAtoms():
                pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())
                symbol = atom.GetSymbol()
                xyz_file.write(f"{symbol} {pos.x} {pos.y} {pos.z}\n")
def write_smiles_to_file(smiles_list, output_file):
    with open(output_file, 'w') as smiles_file:
        for smiles in smiles_list:
            smiles_file.write(f"{smiles}\n")
# Example DataFrame with SMILES strings

# Create .xyz files for each SMILES in the DataFrame
output_folder = 'catxyz_files'
os.makedirs(output_folder, exist_ok=True)
unique_df = df1.drop_duplicates(subset='3,3â€™ Catalyst Substituent ')


for index, row in unique_df.iterrows():
    smiles = row['3,3â€™ Catalyst Substituent ']
    output_file = os.path.join(output_folder, f'{smiles.replace("/", "_")}.xyz')
    smiles_to_xyz(smiles, output_file)


print(f".xyz files created in {output_folder}")
output_file = os.path.join(output_folder, 'catSMILES.txt')
write_smiles_to_file(unique_df['3,3â€™ Catalyst Substituent '], output_file)

[17:09:47] UFFTYPER: Unrecognized atom type: S_6+6 (1)
[17:09:47] UFFTYPER: Unrecognized atom type: S_6+6 (11)


ValueError: Bad Conformer Id

In [5]:

def find_atom_ids(mol, atom_symbols):
    atom_ids = []

    for atom in mol.GetAtoms():
        if atom.GetSymbol() in atom_symbols:
            atom_ids.append(atom.GetIdx() + 1)  # Atom IDs start from 1

    return atom_ids

def find_c_o_atom_ids(smiles,a1='O',a2='C'):
    # Handle cases where there is a '/' in the SMILES string
    #smiles = smiles.replace('/', '')

    mol = Chem.MolFromSmiles(smiles)

    if mol is not None:
        # Add hydrogen atoms
        mol = Chem.AddHs(mol)

        # Find atom IDs for oxygen and carbon in the C=O group with a double bond
        for bond in mol.GetBonds():
            if bond.GetBondTypeAsDouble() == 2.0:
                atom1_symbol = bond.GetBeginAtom().GetSymbol()
                atom2_symbol = bond.GetEndAtom().GetSymbol()

                if atom1_symbol == a1 and atom2_symbol == a2:
                    oxygen_atom_ids = bond.GetBeginAtom().GetIdx()
                    carbon_atom_ids = bond.GetEndAtom().GetIdx()
                    return {'smiles': smiles, 'oxygen_atom_ids': oxygen_atom_ids, 'carbon_atom_ids': carbon_atom_ids}
                elif atom1_symbol == a2 and atom2_symbol == a1:
                    oxygen_atom_ids = bond.GetEndAtom().GetIdx()
                    carbon_atom_ids = bond.GetBeginAtom().GetIdx()
                    return {'smiles': smiles, 'oxygen_atom_ids': oxygen_atom_ids, 'carbon_atom_ids': carbon_atom_ids}

    return None



# Keep only unique SMILES entries
unique_df = df1.drop_duplicates(subset='electrophile SMILES')
# Find atom IDs for oxygen and carbon in the C=O group with a double bond for each unique SMILES
results = []
for index, row in unique_df.iterrows():
    result = find_c_o_atom_ids(row['electrophile SMILES'])
    if result:
        results.append(result)

# Display results
for result in results:
    print(f"SMILES: {result['smiles']}, Oxygen Atom IDs: {result['oxygen_atom_ids']}, Carbon Atom IDs: {result['carbon_atom_ids']}")


SMILES: O=Cc2ccc1ccccc1c2, Oxygen Atom IDs: 0, Carbon Atom IDs: 1
SMILES: COc2ccc1cc(C=O)ccc1c2, Oxygen Atom IDs: 9, Carbon Atom IDs: 8
SMILES: O=Cc2ccc1cc(Br)ccc1c2, Oxygen Atom IDs: 0, Carbon Atom IDs: 1
SMILES: O=Cc1ccccc1, Oxygen Atom IDs: 0, Carbon Atom IDs: 1
SMILES: O=Cc1ccccc1F, Oxygen Atom IDs: 0, Carbon Atom IDs: 1
SMILES: Cc1ccccc1C=O, Oxygen Atom IDs: 8, Carbon Atom IDs: 7
SMILES: Cc1cccc(C=O)c1, Oxygen Atom IDs: 7, Carbon Atom IDs: 6
SMILES: Cc1ccc(C=O)cc1, Oxygen Atom IDs: 6, Carbon Atom IDs: 5
SMILES: CC(C)(C)c1ccc(C=O)cc1, Oxygen Atom IDs: 9, Carbon Atom IDs: 8
SMILES: O=C/C=C/c1ccccc1, Oxygen Atom IDs: 0, Carbon Atom IDs: 1
SMILES: CCCCCCCCCC=O, Oxygen Atom IDs: 10, Carbon Atom IDs: 9
SMILES: CCCCC=O, Oxygen Atom IDs: 5, Carbon Atom IDs: 4
SMILES: O=CCCc1ccccc1, Oxygen Atom IDs: 0, Carbon Atom IDs: 1
SMILES: CC(C)CC=O, Oxygen Atom IDs: 5, Carbon Atom IDs: 4
SMILES: CC(=O)OC1CCCO1, Oxygen Atom IDs: 2, Carbon Atom IDs: 1
SMILES: O=C(OC1CCCO1)c2ccccc2, Oxygen Atom IDs: 0,

# Electrophiles


In [4]:

from rdkit.Chem.rdmolfiles import  MolFromXYZFile
import itertools
import math

def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2 + (point1[2] - point2[2])**2)

def calculate_max_distance(coordinates):
    max_distance = 0.0
    
    # Use itertools to generate all combinations of points
    for pair in itertools.combinations(coordinates, 2):
        distance = calculate_distance(pair[0], pair[1])
        max_distance = max(max_distance, distance)
    
    return max_distance


def calculate_bounding_box_dimensions(coordinates):
    min_x, min_y, min_z = float('inf'), float('inf'), float('inf')
    max_x, max_y, max_z = float('-inf'), float('-inf'), float('-inf')

    for point in coordinates:
        x, y, z = point
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        min_z = min(min_z, z)
        max_x = max(max_x, x)
        max_y = max(max_y, y)
        max_z = max(max_z, z)

    dimensions = [max_x - min_x, max_y - min_y, max_z - min_z]
    dimensions.sort(reverse=True)

    return dimensions

def get_coord(coord_filepath,id):
    mol = MolFromXYZFile(coord_filepath)
    mol.GetConformer()
    positions = []
    for i, atom in enumerate(mol.GetAtoms()):
            positions.append(mol.GetConformer().GetAtomPosition(i))
    max_distance = calculate_max_distance(positions)
    bounding_box_dimensions = calculate_bounding_box_dimensions(positions)
    displace = calculate_distance(mol.GetConformer().GetAtomPosition(id), [x / 2 for x in bounding_box_dimensions])
    return max_distance, bounding_box_dimensions,displace







In [233]:


import os
import subprocess

directory_path = "low_energy"  # Replace with the actual path to your directory
output_file = "results.txt"

# Open the output file in write mode
"""with open(output_file, "w") as file:

    # Iterate over all files in the directory with a .log extension
    for filename in os.listdir(directory_path):
        if filename.endswith(".log"):
            file_path = os.path.join(directory_path, filename)
            
            # Run db.dbstep command for each .log file
            #command = f"db.dbstep {file_path} 1 --commandline --volume --measure classic"
            result = db.dbstep(file_path,1, commandline=True,volume=True,measure='classic',r=5)
            # Use subprocess to run the command
            #result = subprocess.run(command, shell=True, capture_output=True, text=True)
            result = result.bur_vol
            # Write results to the output file
            file.write(f"Results for file {filename}:\n{result}\n")"""



filename_list_file = "Atom_IDs.txt"  # Replace with the actual path to your filename list file

directory_path = "low_energy"  # Replace with the actual path to your directory
output_file = "results35.txt"
# Read the filenames from the specified file
with open(filename_list_file, "r") as filename_list:
    target_filenames = [line.split()[1] for line in filename_list.readlines() if line.strip()]
with open(filename_list_file, "r") as filename_list:
    AIDs= [line.split()[3] for line in filename_list.readlines() if line.strip()]

# Open the output file in write mode
print(AIDs)
with open(output_file, "w") as file:

    # Iterate over the target filenames
    for i in range(len(target_filenames)):
        target_filename = target_filenames[i]
        AID = AIDs[i]
        file_path = os.path.join(directory_path, target_filename)
        
        # Run db.dbstep command for each target file
        
        # Use subprocess to run the command
        result = db.dbstep(file_path,AID, commandline=True,volume=True,measure='classic',r=3.5)
            # Use subprocess to run the command
            #result = subprocess.run(command, shell=True, capture_output=True, text=True)
        result = result.bur_vol
            # Write results to the output file
        file.write(f"Results for file {target_filename}:    {result}\n")


['1', '27', '6', '6', '4', '1', '6', '5', '4', '1', '1', '1', '36', '6', '1', '6', '6', '6', '36', '35', '6', '6', '1', '6', '4', '1', '1', '36', '36', '1', '3', '3', '39']
      R/Å     %V_Bur     %S_Bur
     3.50      53.45       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      42.94       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      45.11       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      45.13       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      58.38       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      53.54       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      45.10       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      53.79       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      36.49       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      53.43       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      53.37       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      53.45       0.00
      R/Å     %V_Bur     %S_Bur
     3.50      45.21       

In [212]:
db.dbstep("low_energy/ph_2hydropyrenemethyl_3_m062x.log",1,commandline=True,volume=True,measure='classic')

      R/Å     %V_Bur     %S_Bur
     3.50      51.15       0.00


In [6]:

def run_dbstep(file_path, atom1, atom2):
    mol = db.dbstep(file_path,atom1,atom2,commandline=True,sterimol=True,measure='classic')  
    mol2 = db.dbstep(file_path,atom2,commandline=True,volume=True,measure='classic')
    #Grab Sterimol Parameters
    L = mol.L
    Bmin = mol.Bmin
    Bmax = mol.Bmax
    bur_shell = mol2.bur_shell
    bur_vol = mol2.bur_vol
    #print(mol2.bur_vol,mol2.bur_shell,mol2.__dict__)
    max_distance, bounding_box_dimensions,disp = get_coord(file_path,atom2)
    tot_V = bounding_box_dimensions[0]*bounding_box_dimensions[1]*bounding_box_dimensions[2]
    max_axis = np.max(bounding_box_dimensions)
    A2 = np.sort(bounding_box_dimensions)[-2]/max_axis
    min_axis = np.min(bounding_box_dimensions)/max_axis
    return L, Bmin,Bmax,bur_shell,bur_vol, max_distance, tot_V, disp,max_axis,A2,min_axis

# Example DataFrame with SMILES strings


# Keep only unique SMILES entries
unique_df = df1.drop_duplicates(subset='electrophile SMILES')

# Find Sterimol parameters for each unique SMILES
results = []
for index, row in unique_df.iterrows():
    c_o_info = find_c_o_atom_ids(row['electrophile SMILES'])
    if c_o_info:
        smiles = c_o_info['smiles']
        oxygen_atom_id = c_o_info['oxygen_atom_ids']
        carbon_atom_id = c_o_info['carbon_atom_ids']

        # Create .xyz file
        output_folder = 'xyz_files'
        os.makedirs(output_folder, exist_ok=True)
        output_file_path = os.path.join(output_folder, f'{smiles.replace("/", "_")}.xyz')
        smiles_to_xyz(smiles, output_file_path)

        # Run db.dbstep and extract Sterimol parameters
        L, Bmin,Bmax,bur_shell,bur_vol, max_distance, tot_V, disp,max_axis,A2,min_axis = run_dbstep(output_file_path, oxygen_atom_id, carbon_atom_id)

        # Extract Sterimol parameters (L, Bmin, Bmax)
        

        results.append({'smiles': smiles, 'L': L, 'Bmin': Bmin, 'Bmax': Bmax, 'bur_shell': bur_shell, 'bur_vol': bur_vol, 'max_distance': max_distance, 'tot_V': tot_V, 'disp': disp, 'max_axis': max_axis, 'A2': A2, 'min_axis': min_axis})

# Display Sterimol results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_csv_file = 'sterimol_results.csv'
results_df.to_csv(results_csv_file, index=False)

NameError: name 'db' is not defined

In [201]:
"""
def run_dbstep(file_path, atom1, atom2):
    mol = db.dbstep(file_path,atom1,atom2,commandline=True,sterimol=True,measure='classic')  
    mol2 = db.dbstep(file_path,atom2,commandline=True,volume=True,measure='classic')
    #Grab Sterimol Parameters
    L = mol.L
    Bmin = mol.Bmin
    Bmax = mol.Bmax
    bur_shell = mol2.bur_shell
    bur_vol = mol2.bur_vol
    #print(mol2.bur_vol,mol2.bur_shell,mol2.__dict__)
    return L, Bmin,Bmax,bur_shell,bur_vol"""

# Example DataFrame with SMILES strings


# Keep only unique SMILES entries
unique_df = df1.drop_duplicates(subset='nucleophile SMILES')

# Find Sterimol parameters for each unique SMILES
results = []
for index, row in unique_df.iterrows():
    c_o_info = find_c_o_atom_ids(row['nucleophile SMILES'],"C","C")
    if c_o_info:
        smiles = c_o_info['smiles']
        oxygen_atom_id = c_o_info['oxygen_atom_ids']
        carbon_atom_id = c_o_info['carbon_atom_ids']

        # Create .xyz file
        output_folder = 'nxyz_files'
        os.makedirs(output_folder, exist_ok=True)
        output_file_path = os.path.join(output_folder, f'{smiles.replace("/", "_")}.xyz')
        smiles_to_xyz(smiles, output_file_path)
        print(smiles)
        # Run db.dbstep and extract Sterimol parameters
        L, Bmin,Bmax,bur_shell,bur_vol, max_distance, tot_V, disp,max_axis,A2,min_axis  = run_dbstep(output_file_path, oxygen_atom_id, carbon_atom_id)

        # Extract Sterimol parameters (L, Bmin, Bmax)
        

        results.append({'smiles': smiles, 'L': L, 'Bmin': Bmin, 'Bmax': Bmax, 'bur_shell': bur_shell, 'bur_vol': bur_vol, 'max_distance': max_distance, 'tot_V': tot_V, 'disp': disp, 'max_axis': max_axis, 'A2': A2, 'min_axis': min_axis})

# Display Sterimol results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_csv_file = 'nsterimol_results.csv'
results_df.to_csv(results_csv_file, index=False)

C=CC[Si](C)(C)C
   nxyz_files/C=CC[Si](C)(C)C.xyz / Bmin:  1.70 / Bmax:  5.51 / L:  6.61
      R/Å     %V_Bur     %S_Bur
     3.50      29.72       0.00
C=C(OC)O[Si](C)(C)C(C)(C)C
   nxyz_files/C=C(OC)O[Si](C)(C)C(C)(C)C.xyz / Bmin:  2.45 / Bmax:  5.79 / L:  7.08
      R/Å     %V_Bur     %S_Bur
     3.50      34.71       0.00
C=C(OCc1ccccc1)O[Si](C)(C)C(C)(C)C
   nxyz_files/C=C(OCc1ccccc1)O[Si](C)(C)C(C)(C)C.xyz / Bmin:  3.18 / Bmax:  7.25 / L:  6.04
      R/Å     %V_Bur     %S_Bur
     3.50      48.73       0.00
C=C(OC1CCCCC1)O[Si](C)(C)C(C)(C)C
   nxyz_files/C=C(OC1CCCCC1)O[Si](C)(C)C(C)(C)C.xyz / Bmin:  3.04 / Bmax:  7.20 / L:  6.26
      R/Å     %V_Bur     %S_Bur
     3.50      43.19       0.00
C=C(O[Si](C)(C)C(C)(C)C)c1ccccc1
   nxyz_files/C=C(O[Si](C)(C)C(C)(C)C)c1ccccc1.xyz / Bmin:  1.88 / Bmax:  6.30 / L:  6.73
      R/Å     %V_Bur     %S_Bur
     3.50      39.54       0.00
C=C(/C=C/c1ccccc1)O[Si](C)(C)C(C)(C)C
   nxyz_files/C=C(_C=C_c1ccccc1)O[Si](C)(C)C(C)(C)C.xyz / Bmin:  1.

In [9]:
results_csv_file = 'sterimol_results.csv'
sterimol_df = pd.read_csv(results_csv_file)
sterimol_df["electrophile SMILES"]=  sterimol_df["smiles"]
# Load additional information from the second CSV file
nresults_csv_file = 'nsterimol_results.csv'
nsterimol_df = pd.read_csv(nresults_csv_file)
nsterimol_df["nucleophile SMILES"]=  nsterimol_df["smiles"]
nsterimol_df["nL"]=  nsterimol_df["L"]
nsterimol_df["nBmin"]=  nsterimol_df["Bmin"]
nsterimol_df["nBmax"]=  nsterimol_df["Bmax"]
nsterimol_df["nbur_vol"]=  nsterimol_df["bur_vol"]
import numpy as np

additional_info_csv_file = 'sterimol/merged.csv'
additional_info_df = pd.read_csv(additional_info_csv_file)
print(len(np.unique(additional_info_df["nucleophile SMILES"])))

# Merge DataFrames based on the 'SMILES' column
merged_df = pd.merge( additional_info_df, sterimol_df, on='electrophile SMILES', how='left')
merged_df = pd.merge( merged_df, nsterimol_df, on='nucleophile SMILES', how='left')

# Save the merged DataFrame to a new CSV file
merged_csv_file = 'merged_results.csv'
merged_df.to_csv(merged_csv_file, index=False)

print(f"Merged results saved to {merged_csv_file}")

50
Merged results saved to merged_results.csv


In [10]:
merged_csv_file = 'merged_results.csv'
nresults_csv_file = 'sterimol/NSMILES_2d_crippen_output.csv'
n2results_csv_file = 'sterimol/NSMILES_2d_output.csv'
eresults_csv_file = 'sterimol/SMILES_2d_crippen_output.csv'
e2results_csv_file = 'sterimol/SMILES_2d_output.csv'
nsterimol_df = pd.read_csv(nresults_csv_file)
n2sterimol_df = pd.read_csv(n2results_csv_file)
esterimol_df = pd.read_csv(eresults_csv_file)
e2sterimol_df = pd.read_csv(e2results_csv_file)
merged_df = pd.read_csv(merged_csv_file)

nsterimol_df["nucleophile SMILES"] =  pd.read_csv('nxyz_files/NSMILES.txt', sep=" ", header=None)
n2sterimol_df["nucleophile SMILES"] =  pd.read_csv('nxyz_files/NSMILES.txt', sep=" ", header=None)
esterimol_df["electrophile SMILES"] =  pd.read_csv('xyz_files/SMILES.txt', sep=" ", header=None)
e2sterimol_df["electrophile SMILES"] =  pd.read_csv('xyz_files/SMILES.txt', sep=" ", header=None)
print(len(np.unique(nsterimol_df["nucleophile SMILES"])))
print(len(np.unique(merged_df["nucleophile SMILES"])))
print(len(np.unique(n2sterimol_df["nucleophile SMILES"])))
print(len(np.unique(e2sterimol_df["electrophile SMILES"])))
print(len(np.unique(esterimol_df["electrophile SMILES"])))
print(len(np.unique(merged_df["electrophile SMILES"])))
merged_df = pd.merge(merged_df, nsterimol_df.drop(["Structure","Property"],axis=1), on='nucleophile SMILES', how='left')
merged_df = pd.merge(merged_df, n2sterimol_df.drop(["Structure","Property"],axis=1), on='nucleophile SMILES', how='left')
merged_df = pd.merge(merged_df, esterimol_df.drop(["Structure","Property"],axis=1), on='electrophile SMILES', how='left')
merged_df = pd.merge(merged_df, e2sterimol_df.drop(["Structure","Property"],axis=1), on='electrophile SMILES', how='left')

merged_csv_file = 'merged_final.csv'
merged_df.to_csv(merged_csv_file, index=False)


50
50
50
151
151
153


In [ ]:
#I have manaully deleted 2 electrophiles